# Assignment 2: Large Language Models for Text Classification

### CS 410/510 Large Language Models Fall 2024

#### Greg Witt

## Load Dataset

**Install Dependencies**

In [ ]:
!pip install datasets

In [24]:
from datasets import load_dataset
import random

# load the training set of tweets
ds_train = load_dataset("cardiffnlp/tweet_sentiment_multilingual", "english", split="train")
# the dataset labels {0 : negative, 1: neutral, 2: positive }
print(f"""tweet_sentiment_multilingual training dataset: 
    -----------------------------------------
        {ds_train} 
    """)
random_tweet_index = random.randint(0,1839)
print(f"""
    Random Tweet:
    {ds_train['text'][random_tweet_index]}

    Label: 
    {ds_train['label'][random_tweet_index] }
""")


tweet_sentiment_multilingual training dataset: 
    -----------------------------------------
        Dataset({
    features: ['text', 'label'],
    num_rows: 1839
}) 
    

    Random Tweet:
    omg jonghyun is following his hyung by giving me 2nd hand embarrassment feels. XD what are you cnblue dorks. 

    Label: 
    0



In [7]:

# load the training set of tweets
ds_validation = load_dataset("cardiffnlp/tweet_sentiment_multilingual", "english", split="validation")
# the dataset labels {0 : negative, 1: neutral, 2: positive }
print(f"""tweet_sentiment_multilingual validation set: 
    -----------------------------------------
        {ds_validation}
    """)
random_tweet_index = random.randint(0,324)
print(f"""
    Random Tweet:
    {ds_validation['text'][7]}

    Label: 
    {ds_validation['label'][7] }
""")


tweet_sentiment_multilingual validation set: 
    -----------------------------------------
        Dataset({
    features: ['text', 'label'],
    num_rows: 324
})
    

    Random Tweet:
    Harper's Worst Offense against Refugees may be Climate Record as rising temperatures add to chaos in the Middle East 

    Label: 
    1



## Load the Models

**Load Dependencies**

In [ ]:

!pip install transformers torch

#### [Lama 3.2 1B](https://huggingface.co/meta-llama/Llama-3.2-1B) 



In [21]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

llama_3_2_1B = "meta-llama/Llama-3.2-1B"

llama_3_2_1B_tokenizer = AutoTokenizer.from_pretrained(llama_3_2_1B)

llama_3_2_1B_model = AutoModelForCausalLM.from_pretrained(llama_3_2_1B)

In [14]:
prompt_text = f"""{tweet} \n Sentiment (positive(2), negative(0), neutral(1)):"""

In [18]:
# tokenizer assigns the probability 
"Text: I hate it! \nSentiment (positive, negative, neutral):"

prompt_approach_two = f"""{tweet} Sentiment: """

In [19]:
# Text to classify
text = "Your text to classify here."

# Tokenize the text
inputs = tokenizer(text, return_tensors="pt")

# Get model predictions
outputs = model(**inputs)
logits = outputs.logits

# Determine the predicted class
predicted_class = torch.argmax(logits, dim=1).item()


NameError: name 'tokenizer' is not defined

In [ ]:
with torch.no_grad():
    outputs = model(**inputs, labels=inputs["input_ids"])
    log_probs = torch.nn.functional.log_softmax(outputs.logits, dim=-1)

In [20]:
Experiment #2 

1 prompt  2 positive , 2 negative, 2 neutral k = 2 {tweet} : Sentiment {?} =>

NameError: name 'Experiment' is not defined

In [23]:
# 3 
google_this if needed.
chain_of_thought = "think about this step by step this sound positive, negative, neurtal?\n"
f"{chain_of_thought} {tweet}"

# emotion based: pull on compassion strings.
"I need this for my job: classify as negative, positive or neutral"

SyntaxError: expected 'else' after 'if' expression (2972846960.py, line 2)